In [ ]:
from google.genai.types import EmbedContentConfig
import concurrent.futures
from google import genai
from google.cloud import bigquery
from google.genai.types import EmbedContentConfig
from datetime import datetime
from src.utils import process_similarity_single_query
import pandas as pd
import pandas_gbq
from sklearn.metrics.pairwise import cosine_similarity
import re
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import normalize, MinMaxScaler, StandardScaler
import logging
import numpy as np
from tqdm import tqdm
from google.cloud.bigquery_storage import BigQueryReadClient
import time
import ast


bigquery_client = bigquery.Client(project='deron-innovations')
bqstorage_client = BigQueryReadClient()
project = "deron-innovations"
location = "us-central1"
client = genai.Client(vertexai=True, project=project, location=location) 

def embed_single_query(input_text):
    response = client.models.embed_content(
        model='gemini-embedding-001',
        contents=input_text,
        config=EmbedContentConfig(task_type='RETRIEVAL_QUERY'),
    )
    return [e.values for e in response.embeddings]

In [2]:
customer_query = "help my order has still not arrived what is going on"

In [5]:
customer_query_emb = embed_single_query(customer_query)

In [6]:
# df_samples_embeddings = add_gemini_embeddings(
#     df=df_sampled,
#     text_column="examples",
#     model='gemini-embedding-001',
#     task_type='RETRIEVAL_DOCUMENT',
#     max_workers=8,
#     batch_size=100)

In [7]:
from chromadb import PersistentClient

CHROMA_PATH = "emb_collection"
COLLECTION = "sample_avg_embeddings_collection"

client = PersistentClient(path=CHROMA_PATH)
col = client.get_collection(COLLECTION)

# Fetch all saved items (ids, documents, metadatas, embeddings)
all_items = col.get(include=["documents", "metadatas", "embeddings"])


df_avg_embeddings_collection = pd.DataFrame({
    'id': all_items['ids'],
    'embedding': all_items['embeddings'].tolist(),
    'document': all_items['documents'],
    'metadatas': all_items['metadatas']
})

# Extract the 'output' value from each metadata dict and create a new column
df_avg_embeddings_collection['output'] = df_avg_embeddings_collection['metadatas'].apply(lambda x: x.get('output') if isinstance(x, dict) else None)
df_avg_embeddings_collection.drop(columns=['id','metadatas'], inplace=True)




CHROMA_PATH = "emb_collection"
COLLECTION = "intent_meaning_collection"

client = PersistentClient(path=CHROMA_PATH)
col = client.get_collection(COLLECTION)

# Fetch all saved items (ids, documents, metadatas, embeddings)
all_items = col.get(include=["documents", "metadatas", "embeddings"])


df_intent_collection = pd.DataFrame({
    'id': all_items['ids'],
    'embedding': all_items['embeddings'].tolist(),
    'document': all_items['documents'],
    'metadatas': all_items['metadatas']
})

# Extract the 'output' value from each metadata dict and create a new column
df_intent_collection['output'] = df_intent_collection['metadatas'].apply(lambda x: x.get('output') if isinstance(x, dict) else None)
df_intent_collection.drop(columns=['id','metadatas'], inplace=True)
df_intent_collection


,embedding,document,output
0,"[-0.010085145942866802, 0.021591518074274063, ...",The customer wants to stop an order from being...,cancel_order
1,"[-0.022726159542798996, -0.0016650878824293613...",The customer wants money back for an order (fu...,refund_request
2,"[0.0022645192220807076, 0.01177155040204525, -...",The customer is experiencing a problem using t...,technical_issue
3,"[-0.025329837575554848, 0.012980042025446892, ...",The customer wants the status or ETA of a ship...,track_order


In [13]:
collections = [
    {
        'name': 'intent_meaning_collection',
        'df': df_intent_collection,          # has columns: output_col, embedding_col, document_col
        'output_col': 'output',
        'embedding_col': 'embedding',
        'document_col': 'document'
    },
    {
        'name': 'sample_avg_embeddings_collection',
        'df': df_avg_embeddings_collection,  # has columns: output_col, embedding_col, document_col
        'output_col': 'output',
        'embedding_col': 'embedding',
        'document_col': 'document'
    }
]

In [16]:
single_result = process_similarity_single_query(
    query_embedding=customer_query_emb,   # list or np.ndarray
    collections=collections,           # same schema as used for batch
    top_n=5
)

In [23]:
from src.utils import select_best_output_across_collections

df_selected = select_best_output_across_collections(
    df=single_result,               # output of batch_process_similarity
    collections=collections, # same list used earlier; only 'name' needed
    criterion='total',   # 'max_top1_margin' | 'max' | 'mean' | 'total'
    normalization='l1'          # 'minmax' | 'zscore' | 'l1' | 'l2' | 'sigmoid | None'
)

In [ ]:
# Create a mapping dictionary for output labels
output_mapping = {
    'cancel_order': 'Cancel Order',
    'refund_request': 'Refund Request',
    'technical_issue': 'Technical Issue',
    'track_order': 'Track Order'
}

# Apply the mapping to create a clean version of the selected output
df_selected['selected_output_clean'] = df_selected['selected_output'].map(output_mapping)
final_prediction = df_selected['selected_output_clean'][0]
final_prediction



0    Track Order
Name: selected_output_clean, dtype: object

In [ ]:
## Visualization of confidence

In [27]:
import src.utils
import importlib
importlib.reload(src.utils)

<module 'src.utils' from 'c:\\Users\\derri\\Documents\\GitHub\\PolyVector-Classifier\\src\\utils.py'>

In [ ]:
import streamlit as st
from src.utils import plot_selected_collection_top5_plotly

# df_selected is the 1-row DataFrame with 'selected_collection' and top-N columns
primary = st.get_option('theme.primaryColor')  # optional, to match app theme
fig, plot_df = plot_selected_collection_top5_plotly(
    df_one_row=df_selected,
    normalization=None,   # or 'softmax'/'minmax'/'l1'/'l2'/'zscore'/'sigmoid'
    top_n=5,
    percent_fmt=True,
    title=None,           # or custom string
    color=primary         # pass theme color for seamless look
)

# st.plotly_chart(fig, use_container_width=True)

In [ ]:
# from src.utils import plot_selected_collection_top5_bar
# import matplotlib.pyplot as plt

# # df_results is your augmented df that includes `selected_collection`

# fig, ax, plot_df = plot_selected_collection_top5_bar(
#     df_selected,   # pass as a one-row DataFrame
#     normalization=None,      # or 'minmax'/'l1'/'l2'/'zscore'/'sigmoid'
#     top_n=5,                      # show top 5
#     percent_fmt=True              # show as percentages
# )
# plt.show()

# # Optional: inspect the data used for the chart
# plot_df